In [77]:
#importing required packages
import requests
import json
import pandas as pd

In [11]:
#posting request to login endpoint with payload as json-string, save response as resp
resp = requests.post('https://ies-midterm.soulution.rocks/login', json={'cuni':'97402897'})
resp

<Response [200]>

In [12]:
#text of the response
resp.text

'{"data": {"personal_code": "a70769c3", "dataset_ids": ["3bd4cabfaa", "288b147f95", "f45ec2e9fc", "aab8ba99c2", "098b83ee51", "64a5d99db0"]}, "message": "Hello, Schubert Paul IN! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook."}'

In [16]:
#looking at the keys of the response
print(resp.json().keys())

dict_keys(['data', 'message'])


In [19]:
#printing personal_code
print(resp.json()['data']['personal_code'])

a70769c3


In [22]:
#saving list of codes
list_codes = resp.json()['data']['dataset_ids']
list_codes

['3bd4cabfaa',
 '288b147f95',
 'f45ec2e9fc',
 'aab8ba99c2',
 '098b83ee51',
 '64a5d99db0']

In [55]:
#get data, store it in the list data
data = []
for codes in range(len(list_codes)):
    data_resp = requests.get(f'https://ies-midterm.soulution.rocks/data/{list_codes[codes]}')
    if data_resp.status_code != 200:
        while data_resp.status_code != 200:
            data_resp = requests.get(f'https://ies-midterm.soulution.rocks/data/{list_codes[codes]}')
    data.append(data_resp.json())

In [177]:
#looking at the keys of all 5 datasets
for set in range(len(data)):
    print(data[set]['data'].keys())

dict_keys(['company', 'data'])
dict_keys(['company', 'data'])
dict_keys(['company', 'data'])
dict_keys(['company', 'data'])
dict_keys(['company', 'data'])
dict_keys(['company', 'data'])


In [178]:
#look at company names and save them in a list
companies = []
for sets in range(len(data)):
    print(data[sets]['data']['company'])
    companies.append(data[sets]['data']['company'])
companies

AUTO
JET
AAL
VOD
MRO
LAND


['AUTO', 'JET', 'AAL', 'VOD', 'MRO', 'LAND']

In [179]:
#get column names, save as list
column_names = list(data[0]['data']['data'][0].keys())
column_names

['Date', 'Open', 'High', 'Low', 'Close', 'Adj.Close', 'Volume']

In [180]:
#setup empty df with column_names
df = pd.DataFrame([])
df

""


In [181]:
#access every column in every entry of every set, but data into a list, add list to df
list_dfs = []
for sets in range(len(data)):
    for entry in range(len(data[sets]['data']['data'])):
        entry_list = []
        #entry_df = pd.DataFrame([[0,0,0,0,0,0,0]], columns = column_names)
        for columns in range(len(column_names)):
            try:
                entry_list.append(data[sets]['data']['data'][entry][column_names[columns]])
            except:
                pass
        entry_series = pd.Series(entry_list)
        df = df.append(entry_series, ignore_index=True)
    list_dfs.append(df)

In [186]:
big_df = pd.concat(list_dfs, keys = companies)
big_df

0                   1                   2  \
AUTO 0     2019-01-02  3.0199999809265137   3.450000047683716   
     1     2019-01-03   3.200000047683716  3.4200000762939453   
     2     2019-01-04  3.4100000858306885  3.4200000762939453   
     3     2019-01-07  3.3499999046325684  3.4200000762939453   
     4     2019-01-08   3.200000047683716   3.299999952316284   
...               ...                 ...                 ...   
LAND 2863  2020-11-17    14.5600004196167  14.843000411987305   
     2864  2020-11-18  14.899999618530273  15.220000267028809   
     2865  2020-11-19  14.770000457763672  14.960000038146973   
     2866  2020-11-20  14.960000038146973  15.100000381469727   
     2867                             NaN                 NaN   

                            3                   4                   5       6  
AUTO 0     3.0199999809265137   3.430000066757202   3.430000066757202   18200  
     1      3.200000047683716  3.3499999046325684  3.3499999046325684   52000  
     2      3.309999942779541  3.3299999237060547  3.3299999237060547   27200  
     3      3.069999933242798  3.1700000762939453  3.1700000762939453   24800  
     4     3.0399999618530273  3.1500000953674316  3.1500000953674316    7300  
...                       ...                 ...                 ...     ...  
LAND 2863  14.520000457763672  14.739999771118164  14.695181846618652  298700  
     2864  14.779999732971191  14.800000190734863  14.755000114440918  351800  
     2865    14.6899995803833  14.960000038146973  14.960000038146973  256100  
     2866  14.850000381469727  15.029999732971191  15.029999732971191  195800  
     2867                 NaN                 NaN                 NaN     NaN  

[10029 rows x 7 columns]

In [187]:
#got the df with right column names; problem: no index for where which dataset ends
big_df.columns = column_names
big_df

Date                Open                High  \
AUTO 0     2019-01-02  3.0199999809265137   3.450000047683716   
     1     2019-01-03   3.200000047683716  3.4200000762939453   
     2     2019-01-04  3.4100000858306885  3.4200000762939453   
     3     2019-01-07  3.3499999046325684  3.4200000762939453   
     4     2019-01-08   3.200000047683716   3.299999952316284   
...               ...                 ...                 ...   
LAND 2863  2020-11-17    14.5600004196167  14.843000411987305   
     2864  2020-11-18  14.899999618530273  15.220000267028809   
     2865  2020-11-19  14.770000457763672  14.960000038146973   
     2866  2020-11-20  14.960000038146973  15.100000381469727   
     2867                             NaN                 NaN   

                          Low               Close           Adj.Close  Volume  
AUTO 0     3.0199999809265137   3.430000066757202   3.430000066757202   18200  
     1      3.200000047683716  3.3499999046325684  3.3499999046325684   52000  
     2      3.309999942779541  3.3299999237060547  3.3299999237060547   27200  
     3      3.069999933242798  3.1700000762939453  3.1700000762939453   24800  
     4     3.0399999618530273  3.1500000953674316  3.1500000953674316    7300  
...                       ...                 ...                 ...     ...  
LAND 2863  14.520000457763672  14.739999771118164  14.695181846618652  298700  
     2864  14.779999732971191  14.800000190734863  14.755000114440918  351800  
     2865    14.6899995803833  14.960000038146973  14.960000038146973  256100  
     2866  14.850000381469727  15.029999732971191  15.029999732971191  195800  
     2867                 NaN                 NaN                 NaN     NaN  

[10029 rows x 7 columns]

In [ ]:
#okay indexes and column names are there